In [4]:
 import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [5]:
jobs_df = pd.read_csv('../data/job_postings.csv')
skills_df = pd.read_csv('../data/job_skills.csv')
summary_df = pd.read_csv('../data/job_summary.csv')


In [6]:
jobs_df.head()
skills_df.head()
summary_df.head()


,job_link,job_summary
0,https://www.linkedin.com/jobs/view/senior-mach...,Company Description\nJobs for Humanity is part...
1,https://www.linkedin.com/jobs/view/principal-s...,Who We Are\nAurora (Nasdaq: AUR) is delivering...
2,https://www.linkedin.com/jobs/view/senior-etl-...,"Location: New York City, NY\nPosition Summary\..."
3,https://www.linkedin.com/jobs/view/senior-data...,Responsibilities:\nCandidate must have signifi...
4,https://www.linkedin.com/jobs/view/lead-data-e...,Dice is the leading career destination for tec...


In [7]:
jobs_df.columns


Index(['job_link', 'last_processed_time', 'last_status', 'got_summary',
       'got_ner', 'is_being_worked', 'job_title', 'company', 'job_location',
       'first_seen', 'search_city', 'search_country', 'search_position',
       'job_level', 'job_type'],
      dtype='object')

In [8]:
skills_df.columns


Index(['job_link', 'job_skills'], dtype='object')

In [9]:
summary_df.columns


Index(['job_link', 'job_summary'], dtype='object')

In [10]:
jobs_df.columns = jobs_df.columns.str.lower().str.strip()
skills_df.columns = skills_df.columns.str.lower().str.strip()
summary_df.columns = summary_df.columns.str.lower().str.strip()


In [11]:
jobs_df[['job_link']].head()


,job_link
0,https://www.linkedin.com/jobs/view/senior-mach...
1,https://www.linkedin.com/jobs/view/principal-s...
2,https://www.linkedin.com/jobs/view/senior-etl-...
3,https://www.linkedin.com/jobs/view/senior-data...
4,https://www.linkedin.com/jobs/view/lead-data-e...


In [12]:
jobs_df[['job_link']].head()


,job_link
0,https://www.linkedin.com/jobs/view/senior-mach...
1,https://www.linkedin.com/jobs/view/principal-s...
2,https://www.linkedin.com/jobs/view/senior-etl-...
3,https://www.linkedin.com/jobs/view/senior-data...
4,https://www.linkedin.com/jobs/view/lead-data-e...


In [13]:
summary_df[['job_link']].head()


,job_link
0,https://www.linkedin.com/jobs/view/senior-mach...
1,https://www.linkedin.com/jobs/view/principal-s...
2,https://www.linkedin.com/jobs/view/senior-etl-...
3,https://www.linkedin.com/jobs/view/senior-data...
4,https://www.linkedin.com/jobs/view/lead-data-e...


In [14]:
jobs_merged = jobs_df.merge(
    summary_df[['job_link', 'job_summary']],
    on='job_link',
    how='left'
)


In [15]:
jobs_merged[['job_link', 'job_summary']].head()


,job_link,job_summary
0,https://www.linkedin.com/jobs/view/senior-mach...,Company Description\nJobs for Humanity is part...
1,https://www.linkedin.com/jobs/view/principal-s...,Who We Are\nAurora (Nasdaq: AUR) is delivering...
2,https://www.linkedin.com/jobs/view/senior-etl-...,"Location: New York City, NY\nPosition Summary\..."
3,https://www.linkedin.com/jobs/view/senior-data...,Responsibilities:\nCandidate must have signifi...
4,https://www.linkedin.com/jobs/view/lead-data-e...,Dice is the leading career destination for tec...


In [16]:
jobs_merged.to_csv('../data/cleaned_jobs.csv', index=False)


In [17]:
# Clean job summary text
jobs_merged['job_summary'] = (
    jobs_merged['job_summary']
    .astype(str)
    .str.lower()
    .str.replace(r'\n', ' ', regex=True)
    .str.replace(r'[^a-z0-9 ]', ' ', regex=True)
)

# Skill dictionary
skill_keywords = {
    'python': ['python'],
    'sql': ['sql', 'mysql', 'postgres', 'sqlite'],
    'excel': ['excel'],
    'power_bi': ['power bi', 'powerbi'],
    'tableau': ['tableau'],
    'machine_learning': ['machine learning', 'ml'],
    'statistics': ['statistics'],
    'cloud': ['aws', 'azure', 'gcp']
}

# Create skill flags
for skill, keywords in skill_keywords.items():
    jobs_merged[f'has_{skill}'] = jobs_merged['job_summary'].apply(
        lambda text: int(any(k in text for k in keywords))
    )


In [18]:
jobs_merged.filter(like='has_').head()


,has_python,has_sql,has_excel,has_power_bi,has_tableau,has_machine_learning,has_statistics,has_cloud
0,1,0,0,0,0,1,0,1
1,1,0,0,0,0,1,0,0
2,0,1,1,0,0,0,0,1
3,1,1,0,0,0,0,0,1
4,1,1,0,0,0,1,0,1


In [19]:
jobs_merged.to_csv('../data/cleaned_jobs.csv', index=False)
